In [ ]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Additional library for handling dynamic content, such as Google ad pop-ups
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# Configure the Chrome WebDriver options for headless execution if preferred
chrome_options = Options()
chrome_options.add_argument("--headless")  # Comment this line to see browser actions

# Start the Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [ ]:
# Cell 2: Define Helper Functions for Data Formatting and Error Handling

# Function to format state to abbreviations
def format_state(state_name):
    # Dictionary of state abbreviations
    state_abbr = {
        "California": "CA",
        "Colorado": "CO",
        # Add other state mappings as needed
    }
    return state_abbr.get(state_name, state_name)  # Default to state_name if not found

# Function to format phone numbers to (XXX) XXX-XXXX
def format_phone(phone):
    phone_digits = re.sub(r"\D", "", phone)  # Remove non-digit characters
    return f"({phone_digits[:3]}) {phone_digits[3:6]}-{phone_digits[6:10]}" if len(phone_digits) >= 10 else phone
